In [1]:
import torch
import torch.nn.functional as F

def mock_prob_distribution(input_tensor: torch.Tensor) -> torch.Tensor:

    batch_size = input_tensor.size(0)
    prob_dist = torch.rand(batch_size, 151246)  # Random values in [0, 1)
    prob_dist = F.softmax(prob_dist, dim=1)    # Normalize to sum to 1 per row
    return prob_dist

In [2]:
tensor = torch.tensor([
    [1, 2, 151643, 151643, 151643],
    [3, 151643, 151643, 151643, 151643],
    [5, 6, 7, 8, 9],  # 无151643的行
    [151643, 151643, 151643, 151643, 151643]
])



In [3]:
mask = (tensor==151643)

In [4]:
mask

tensor([[False, False,  True,  True,  True],
        [False,  True,  True,  True,  True],
        [False, False, False, False, False],
        [ True,  True,  True,  True,  True]])

In [6]:
first_pad_indices = mask.int().argmax(dim=1)

In [7]:
first_pad_indices


tensor([2, 1, 0, 0])

In [8]:
no_pad_mask = ~mask.any(dim=1)
no_pad_mask

tensor([False, False,  True, False])

In [9]:
first_pad_indices[no_pad_mask] = tensor.size(1)

In [10]:
first_pad_indices


tensor([2, 1, 5, 0])

In [11]:
tensor

tensor([[     1,      2, 151643, 151643, 151643],
        [     3, 151643, 151643, 151643, 151643],
        [     5,      6,      7,      8,      9],
        [151643, 151643, 151643, 151643, 151643]])

In [12]:
max_seq_len = tensor.size(1)+100
bsz = 8
mask = (tensor == 151643)
first_pad_indices = mask.int().argmax(dim=1)
no_mask = ~mask.any(dim=1)
first_pad_indices[no_mask] = tensor.size(1)
min_prompt_len = first_pad_indices.min()
max_prompt_len = first_pad_indices.max()

total_len = max_seq_len
pad_id = 151643
tokens = tensor

token_logprobs = torch.zeros_like(tokens, dtype=torch.float)

prev_pos = 0
eos_reached = torch.tensor([False] * bsz, device="cuda")
input_text_mask = tokens != pad_id

In [13]:
input_text_mask


tensor([[ True,  True, False, False, False],
        [ True, False, False, False, False],
        [ True,  True,  True,  True,  True],
        [False, False, False, False, False]])

In [14]:
tensor

tensor([[     1,      2, 151643, 151643, 151643],
        [     3, 151643, 151643, 151643, 151643],
        [     5,      6,      7,      8,      9],
        [151643, 151643, 151643, 151643, 151643]])

In [17]:
next_token = torch.tensor([0]*4)
next_token = torch.where(input_text_mask[:,4],tensor[:,4],next_token)
next_token

tensor([0, 0, 9, 0])